<a href="https://colab.research.google.com/github/Priya9112/Null-Class-Data-Science-Internship-/blob/main/French_Tamil_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
import collections
import numpy as np
import json

from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from keras.models import Model, Sequential
from keras.layers import Input, Dense, Embedding, GRU, LSTM, Bidirectional, Dropout, Activation, TimeDistributed, RepeatVector
from keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy

In [ ]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 2699494265312168631
xla_global_id: -1
]


In [ ]:
def load_data(path):
    input_file = path
    with open(input_file, "r", encoding='utf-8') as f:
        data = f.read()
    return data.split('\n')

french_sentences = load_data('/content/drive/MyDrive/french-tamil/fr.txt')
tamil_sentences = load_data('/content/drive/MyDrive/french-tamil/ta.txt')

In [ ]:

french_sentences[:5]

['À la moitié du 16e siècle, les Italiens étaient fascinés par un type de chanteur masculin dont le registre incroyable contenait des notes auparavant considérées comme impossibles pour les hommes.',
 'Néanmoins, ce don coûtait cher.',
 'Pour empêcher que leur voix se casse, ces chanteurs étaient castrés avant leur puberté, freinant les processus hormonaux qui rendraient leur voix grave.',
 "Appelés castrats, leur voix douce et angélique était célèbre en Europe, jusqu'à ce que l'opération cruelle soit proscrite vers l'an 1800.",
 "Bien qu'une voix d'enfant accède à un registre musical extraordinaire, une voix développée naturellement est à même d'avoir une variété incroyable."]

In [ ]:
french_words_counter = collections.Counter([word for sentence in french_sentences for word in sentence.split()])
tamil_words_counter = collections.Counter([word for sentence in tamil_sentences for word in sentence.split()])

print('{} French words.'.format(len([word for sentence in french_sentences for word in sentence.split()])))
print('{} unique French words.'.format(len(french_words_counter)))
print('10 Most common words in the French dataset:')
print('"' + '" "'.join(list(zip(*french_words_counter.most_common(10)))[0]) + '"')

print()
print('{} Tamil words.'.format(len([word for sentence in tamil_sentences for word in sentence.split()])))
print('{} unique Tamil words.'.format(len(tamil_words_counter)))
print('10 Most common words in the Tamil dataset:')
print('"' + '" "'.join(list(zip(*tamil_words_counter.most_common(10)))[0]) + '"')

7152230 French words.
225805 unique French words.
10 Most common words in the French dataset:
"de" "la" "et" "les" "que" "à" "le" "des" "un" "en"

129862 Tamil words.
41275 unique Tamil words.
10 Most common words in the Tamil dataset:
"ஒரு" "நான்" "இந்த" "நாம்" "இது" "அது" "நீங்கள்" "ஆனால்" "என்று" "மற்றும்"


In [ ]:
from keras.preprocessing.text import Tokenizer
import numpy as np

# French sentences
french_sentences = [
    'Le renard brun saute par-dessus le chien paresseux.',
    'Par Jove, mon étude rapide de lexicographie a gagné un prix.',
    'Ceci est une phrase courte.']

# Tamil sentences
tamil_sentences = [
    'அந்த கருமை நரி கண்டது ஓயாத நாயை தாண்டியது.',
    'ஜோவேயின் கீழ், என் விரைவான இலக்கியக் கல்வி பரிசை வென்றது.',
    'இது ஒரு குறுகிய வாக்கியம்.']

def tokenize(x):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(x)
    return tokenizer.texts_to_sequences(x), tokenizer

french_tokenized, french_tokenizer = tokenize(french_sentences)
tamil_tokenized, tamil_tokenizer = tokenize(tamil_sentences)

print(french_tokenizer.word_index)
print()
for sample_i, (sent, token_sent) in enumerate(zip(french_sentences, french_tokenized)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(sent))
    print('  Output: {}'.format(token_sent))

print()
print(tamil_tokenizer.word_index)
print()
for sample_i, (sent, token_sent) in enumerate(zip(tamil_sentences, tamil_tokenized)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(sent))
    print('  Output: {}'.format(token_sent))

{'le': 1, 'par': 2, 'renard': 3, 'brun': 4, 'saute': 5, 'dessus': 6, 'chien': 7, 'paresseux': 8, 'jove': 9, 'mon': 10, 'étude': 11, 'rapide': 12, 'de': 13, 'lexicographie': 14, 'a': 15, 'gagné': 16, 'un': 17, 'prix': 18, 'ceci': 19, 'est': 20, 'une': 21, 'phrase': 22, 'courte': 23}

Sequence 1 in x
  Input:  Le renard brun saute par-dessus le chien paresseux.
  Output: [1, 3, 4, 5, 2, 6, 1, 7, 8]
Sequence 2 in x
  Input:  Par Jove, mon étude rapide de lexicographie a gagné un prix.
  Output: [2, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]
Sequence 3 in x
  Input:  Ceci est une phrase courte.
  Output: [19, 20, 21, 22, 23]

{'அந்த': 1, 'கருமை': 2, 'நரி': 3, 'கண்டது': 4, 'ஓயாத': 5, 'நாயை': 6, 'தாண்டியது': 7, 'ஜோவேயின்': 8, 'கீழ்': 9, 'என்': 10, 'விரைவான': 11, 'இலக்கியக்': 12, 'கல்வி': 13, 'பரிசை': 14, 'வென்றது': 15, 'இது': 16, 'ஒரு': 17, 'குறுகிய': 18, 'வாக்கியம்': 19}

Sequence 1 in x
  Input:  அந்த கருமை நரி கண்டது ஓயாத நாயை தாண்டியது.
  Output: [1, 2, 3, 4, 5, 6, 7]
Sequence 2 in x
  Input

In [ ]:
from keras.preprocessing.sequence import pad_sequences
import numpy as np

def pad(x, length=None):
    if length is None:
        length = max([len(sentence) for sentence in x])
    return pad_sequences(x, maxlen=length, padding='post')

french_padded = pad(french_tokenized)
tamil_padded = pad(tamil_tokenized)

for sample_i, (token_sent, pad_sent) in enumerate(zip(french_tokenized, french_padded)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(np.array(token_sent)))
    print('  Output: {}'.format(pad_sent))

for sample_i, (token_sent, pad_sent) in enumerate(zip(tamil_tokenized, tamil_padded)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(np.array(token_sent)))
    print('  Output: {}'.format(pad_sent))

Sequence 1 in x
  Input:  [1 3 4 5 2 6 1 7 8]
  Output: [1 3 4 5 2 6 1 7 8 0 0]
Sequence 2 in x
  Input:  [ 2  9 10 11 12 13 14 15 16 17 18]
  Output: [ 2  9 10 11 12 13 14 15 16 17 18]
Sequence 3 in x
  Input:  [19 20 21 22 23]
  Output: [19 20 21 22 23  0  0  0  0  0  0]
Sequence 1 in x
  Input:  [1 2 3 4 5 6 7]
  Output: [1 2 3 4 5 6 7 0]
Sequence 2 in x
  Input:  [ 8  9 10 11 12 13 14 15]
  Output: [ 8  9 10 11 12 13 14 15]
Sequence 3 in x
  Input:  [16 17 18 19]
  Output: [16 17 18 19  0  0  0  0]


In [ ]:
def preprocess(x,y):
    preprocess_x, x_tk = tokenize(x)
    preprocess_y, y_tk = tokenize(y)

    preprocess_x = pad(preprocess_x)
    preprocess_y = pad(preprocess_y)

    preprocess_y = preprocess_y.reshape(*preprocess_y.shape, 1)

    return preprocess_x, preprocess_y, x_tk, y_tk

preproc_french_sentences, preproc_tamil_sentences, french_tokenizer, tamil_tokenizer = preprocess(french_sentences, tamil_sentences)

max_french_sequence_length = preproc_french_sentences.shape[1]
max_tamil_sequence_length = preproc_tamil_sentences.shape[1]
french_vocab_size = len(french_tokenizer.word_index)
tamil_vocab_size = len(tamil_tokenizer.word_index)

print('Data Preprocessed')
print("Max French sentence length:", max_french_sequence_length)
print("Max Tamil sentence length:", max_tamil_sequence_length)
print("French vocabulary size:", french_vocab_size)
print("Tamil vocabulary size:", tamil_vocab_size)

Data Preprocessed
Max French sentence length: 11
Max Tamil sentence length: 8
French vocabulary size: 23
Tamil vocabulary size: 19


In [ ]:
def logits_to_text(logits, tokenizer):
    index_to_words = {id: word for word, id in tokenizer.word_index.items()}
    index_to_words[0] = '<PAD>'

    return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, TimeDistributed, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import sparse_categorical_crossentropy
import numpy as np

def simple_model(input_shape, output_sequence_length, french_vocab_size, tamil_vocab_size):
    learning_rate = 0.005
    model = Sequential()
    model.add(GRU(256, input_shape=input_shape[1:], return_sequences=True))
    model.add(TimeDistributed(Dense(1024, activation='relu')))
    model.add(Dropout(0.5))
    model.add(TimeDistributed(Dense(tamil_vocab_size, activation='softmax')))
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])

    return model

preproc_tamil_sentences = np.maximum(preproc_tamil_sentences - 1, 0)
tmp_x = pad(preproc_french_sentences, max_tamil_sequence_length)
tmp_x = tmp_x.reshape((-1, preproc_tamil_sentences.shape[-2], 1))

simple_rnn_model = simple_model(
    tmp_x.shape,
    max_tamil_sequence_length,
    french_vocab_size,
    tamil_vocab_size
)

simple_rnn_model.fit(tmp_x, preproc_tamil_sentences, batch_size=1024, epochs=10, validation_split=0.2)


Epoch 1/10
1/1 [==============================] - 3s 3s/step - loss: 2.9242 - accuracy: 0.0625 - val_loss: 3.7751 - val_accuracy: 0.0000e+00
Epoch 2/10
1/1 [==============================] - 0s 47ms/step - loss: 2.7888 - accuracy: 0.1875 - val_loss: 4.1709 - val_accuracy: 0.5000
Epoch 3/10
1/1 [==============================] - 0s 42ms/step - loss: 2.4623 - accuracy: 0.1250 - val_loss: 4.5870 - val_accuracy: 0.5000
Epoch 4/10
1/1 [==============================] - 0s 45ms/step - loss: 2.6174 - accuracy: 0.1875 - val_loss: 4.9113 - val_accuracy: 0.5000
Epoch 5/10
1/1 [==============================] - 0s 42ms/step - loss: 2.5661 - accuracy: 0.0625 - val_loss: 5.2899 - val_accuracy: 0.5000
Epoch 6/10
1/1 [==============================] - 0s 43ms/step - loss: 2.2485 - accuracy: 0.3125 - val_loss: 5.6613 - val_accuracy: 0.5000
Epoch 7/10
1/1 [==============================] - 0s 44ms/step - loss: 2.4285 - accuracy: 0.1875 - val_loss: 6.0531 - val_accuracy: 0.5000
Epoch 8/10
1/1 [=========

In [ ]:
print("Prediciton:")
print(logits_to_text(simple_rnn_model.predict(tmp_x[:1])[0], tamil_tokenizer))

print("\nCorrect Translation:")
print(tamil_sentences[:1])

print('\nOriginal text:')
print(french_sentences[:1])

Prediciton:
1/1 [==============================] - 0s 410ms/step
தாண்டியது நரி ஓயாத நாயை <PAD> ஓயாத <PAD> <PAD>

Correct Translation:
['அந்த கருமை நரி கண்டது ஓயாத நாயை தாண்டியது.']

Original text:
['Le renard brun saute par-dessus le chien paresseux.']


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Bidirectional, GRU, TimeDistributed, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import sparse_categorical_crossentropy
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

def bd_model(input_shape, tamil_vocab_size):
    learning_rate = 0.005

    model = Sequential()
    model.add(Bidirectional(GRU(128, return_sequences=True), input_shape=input_shape[1:]))
    model.add(TimeDistributed(Dense(1024, activation='relu')))
    model.add(Dropout(0.5))
    model.add(TimeDistributed(Dense(tamil_vocab_size + 1, activation='softmax')))

    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])

    return model

french_tokenized = [[1, 2, 3], [4, 5, 6, 7, 8], [9, 10, 11, 12, 13, 14]]
tamil_padded = [[1, 2, 3, 4, 5, 6, 7, 8], [4, 5, 6, 7, 8, 9, 10, 11], [9, 10, 11, 12, 13, 14, 15, 16]]

max_tamil_sequence_length = max(len(seq) for seq in tamil_padded)
tmp_x = pad_sequences(french_tokenized, maxlen=max_tamil_sequence_length, padding='post')
tamil_padded = pad_sequences(tamil_padded, maxlen=max_tamil_sequence_length, padding='post')
tamil_padded = np.array(tamil_padded) - 1
print("Shape of tmp_x before reshape:", tmp_x.shape)
print("Shape of tamil_padded:", tamil_padded.shape)
tmp_x = tmp_x.reshape((tamil_padded.shape[0], tmp_x.shape[1], 1))

tamil_vocab_size = max(np.max(tamil_padded), 0) + 1

bd_rnn_model = bd_model(
    tmp_x.shape,
    tamil_vocab_size)

print(bd_rnn_model.summary())

bd_rnn_model.fit(tmp_x, tamil_padded, batch_size=1024, epochs=10, validation_split=0.2)


Shape of tmp_x before reshape: (3, 8)
Shape of tamil_padded: (3, 8)
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional (Bidirection  (None, 8, 256)            100608    
 al)                                                             
                                                                 
 time_distributed_2 (TimeDi  (None, 8, 1024)           263168    
 stributed)                                                      
                                                                 
 dropout_1 (Dropout)         (None, 8, 1024)           0         
                                                                 
 time_distributed_3 (TimeDi  (None, 8, 17)             17425     
 stributed)                                                      
                                                                 
Total params: 381201 (1.45 MB)
Trainable params: 381

In [ ]:
print("Prediciton:")
print(logits_to_text(bd_rnn_model.predict(tmp_x[:1])[0], tamil_tokenizer))

print("\nCorrect Translation:")
print(tamil_sentences[:1])

print('\nOriginal text:')
print(french_sentences[:1])

Prediciton:
1/1 [==============================] - 1s 771ms/step
நரி நரி தாண்டியது தாண்டியது தாண்டியது தாண்டியது தாண்டியது தாண்டியது

Correct Translation:
['அந்த கருமை நரி கண்டது ஓயாத நாயை தாண்டியது.']

Original text:
['Le renard brun saute par-dessus le chien paresseux.']


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Bidirectional, GRU, TimeDistributed, Dense, Dropout, Embedding
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import sparse_categorical_crossentropy
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

def bidirectional_embed_model(input_shape, french_vocab_size, tamil_vocab_size):
    learning_rate = 0.005

    model = Sequential()
    model.add(Embedding(french_vocab_size, 256, input_length=input_shape[1]))
    model.add(Bidirectional(GRU(256, return_sequences=True)))
    model.add(TimeDistributed(Dense(1024, activation='relu')))
    model.add(Dropout(0.5))
    model.add(TimeDistributed(Dense(tamil_vocab_size, activation='softmax')))

    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])

    return model

french_tokenized = [[1, 2, 3], [4, 5, 6, 7, 8], [9, 10, 11, 12, 13, 14]]
tamil_padded = [[1, 2, 3, 4, 5, 6, 7, 8], [4, 5, 6, 7, 8, 9, 10, 11], [9, 10, 11, 12, 13, 14, 15, 16]]

max_tamil_sequence_length = max(len(seq) for seq in tamil_padded)
tmp_x = pad_sequences(french_tokenized, maxlen=max_tamil_sequence_length, padding='post')
tamil_padded = pad_sequences(tamil_padded, maxlen=max_tamil_sequence_length, padding='post')
tamil_padded = np.array(tamil_padded) - 1
print("Shape of tmp_x before reshape:", tmp_x.shape)
print("Shape of tamil_padded:", tamil_padded.shape)
tmp_x = np.array(tmp_x)
french_vocab_size = max(max(seq) for seq in french_tokenized) + 1
tamil_vocab_size = max(np.max(tamil_padded), 0) + 1

embed_rnn_model = bidirectional_embed_model(
    tmp_x.shape,
    french_vocab_size,
    tamil_vocab_size)

print(embed_rnn_model.summary())

embed_rnn_model.fit(tmp_x, tamil_padded, batch_size=1024, epochs=10, validation_split=0.2)


Shape of tmp_x before reshape: (3, 8)
Shape of tamil_padded: (3, 8)
Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 8, 256)            3840      
                                                                 
 bidirectional_1 (Bidirecti  (None, 8, 512)            789504    
 onal)                                                           
                                                                 
 time_distributed_4 (TimeDi  (None, 8, 1024)           525312    
 stributed)                                                      
                                                                 
 dropout_2 (Dropout)         (None, 8, 1024)           0         
                                                                 
 time_distributed_5 (TimeDi  (None, 8, 16)             16400     
 stributed)                                         

In [ ]:
print("Prediciton:")
print(logits_to_text(embed_rnn_model.predict(tmp_x[:1])[0], tamil_tokenizer))

print("\nCorrect Translation:")
print(tamil_sentences[:1])

print('\nOriginal text:')
print(french_sentences[:1])

Prediciton:
1/1 [==============================] - 1s 710ms/step
<PAD> அந்த கருமை நரி கண்டது ஓயாத நாயை தாண்டியது

Correct Translation:
['அந்த கருமை நரி கண்டது ஓயாத நாயை தாண்டியது.']

Original text:
['Le renard brun saute par-dessus le chien paresseux.']


In [ ]:
import json

embed_rnn_model.save('/content/drive/MyDrive/french_to_tamil_model')

with open('/content/drive/MyDrive/ft-json/french_tokenizer.json', 'w', encoding='utf8') as f:
    f.write(json.dumps(french_tokenizer.to_json(), ensure_ascii=False))

with open('/content/drive/MyDrive/ft-json/tamil_tokenizer.json', 'w', encoding='utf8') as f:
    f.write(json.dumps(tamil_tokenizer.to_json(), ensure_ascii=False))

max_tamil_sequence_length_json = max_tamil_sequence_length
with open('/content/drive/MyDrive/ft-json/sequence_length.json', 'w', encoding='utf8') as f:
    f.write(json.dumps(max_tamil_sequence_length_json, ensure_ascii=False))
